###  CNN MicroGrad demo

In [2]:
from micrograd.engine import Value
from micrograd.nn import Convolution, CNN, MLP, Neuron
from keras.datasets import mnist
import numpy as np

2023-12-01 14:54:27.393728: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-01 14:54:28.143559: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-12-01 14:54:28.143583: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-12-01 14:54:30.456362: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()

In [4]:
train_X = np.expand_dims(train_X, axis=1)
test_X = np.expand_dims(test_X, axis=1)
train_X.shape

(60000, 1, 28, 28)

In [5]:
mlp = MLP(1250, [64, 10])
my_cnn = CNN([Convolution(1, 3, kernel_size=3), Convolution(3, 2, kernel_size=2)], mlp, is_softmax=True)
out = my_cnn(train_X[:3])
# my_conv = Convolution(1, 2, kernel_size=3)
# my_output = my_conv(train_X[:100])
# print(my_output.shape)

AssertionError: input shape to neuron mismatch, expected 1250 but got 3750

In [5]:
np.sum(out)

Value(data=1.0000000000000002, grad=0)